<a href="https://colab.research.google.com/github/elizabethavargas/baseball/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bat-Level Baseball Data

### Download Data

In [ ]:
# imports
!pip install pybaseball
from pybaseball import statcast
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.6 MB/s eta 0:00:00


In [ ]:
# download data
start_date = "2024-01-01"
end_date = "2024-12-31"
data = statcast(start_dt=start_date, end_dt=end_date)

# filter data to regular games in the 2024 MLB season
data = data[data.game_type == 'R']
data

This is a large query, it may take a moment to complete


/usr/local/lib/python3.12/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


  0%|          | 0/246 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = 

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
200,FF,2024-09-30,97.4,-2.1,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,...,1,1.4,0.96,0.96,17.6,6.149605,12.090516,22.1604,45.805662,22.048373
203,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,NaN,ball,...,1,2.14,-0.2,-0.2,23.1,<NA>,<NA>,<NA>,<NA>,<NA>
209,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,1,2.37,-0.12,-0.12,22.4,23.541699,-27.093819,34.778701,45.227965,45.368412
215,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,NaN,ball,...,1,2.09,-0.21,-0.21,22.4,<NA>,<NA>,<NA>,<NA>,<NA>
221,SL,2024-09-30,89.1,-2.13,5.15,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,1,2.2,-0.17,-0.17,20.2,23.112048,-30.629825,33.038132,53.011806,51.686541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,FF,2024-03-20,95.7,-2.13,5.77,"Darvish, Yu",660271,506433,NaN,ball,...,1,1.03,0.54,-0.54,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2190,SI,2024-03-20,93.9,-2.17,5.7,"Darvish, Yu",605141,506433,NaN,ball,...,1,1.57,1.34,1.34,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2360,SI,2024-03-20,93.4,-2.0,5.7,"Darvish, Yu",605141,506433,NaN,ball,...,1,1.63,1.42,1.42,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2422,FF,2024-03-20,92.6,-1.94,5.83,"Darvish, Yu",605141,506433,NaN,foul,...,1,1.11,0.25,0.25,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
# keep relevant columns
df = data[['game_date', 'game_pk', 'batter', 'pitcher', 'launch_speed',
           'launch_angle', 'age_bat', 'events', 'home_team',
           'stand', 'p_throws', 'pitch_type', 'release_speed']].copy()

# keep rows for everything that's NOT a non-batted-ball event
non_batted_ball_events = ["strikeout", "strikeout_double_play", "walk",
    "hit_by_pitch", "catcher_interf", "truncated_pa", "sac_fly", "sac_bunt"]

df = df[~df["events"].isin(non_batted_ball_events)]

# drop na for rows without launch data
df.dropna(subset=['launch_speed', 'launch_angle', "events"], inplace=True)
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,stand,p_throws,pitch_type,release_speed
200,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,R,R,FF,97.4
257,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,R,R,SL,90.4
291,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,L,R,FF,94.2
400,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,R,R,FF,95.2
452,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,L,R,CU,86.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,L,R,SI,93.9
1757,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,R,R,FF,96.6
1899,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,R,R,FF,96.9
1173,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,R,R,FF,94.5


In [ ]:
# combine hand pitcher threw with and the hand the batter hit with into one column (1 if same, 0 if different)
df['pitcher_batter_handedness'] = (df["p_throws"] == df["stand"]).astype(int)

# Encode home_run as binary target
df['home_run'] = (df['events'] == 'home_run').astype(int)
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,stand,p_throws,pitch_type,release_speed,pitcher_batter_handedness,home_run
200,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,R,R,FF,97.4,1,0
257,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,R,R,SL,90.4,1,0
291,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,L,R,FF,94.2,0,0
400,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,R,R,FF,95.2,1,0
452,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,L,R,CU,86.4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,L,R,SI,93.9,0,0
1757,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,R,R,FF,96.6,1,0
1899,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,R,R,FF,96.9,1,0
1173,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,R,R,FF,94.5,1,0


### Add Altitide, Weather, Time, and Field Data

In [ ]:
import os, time, json, requests
import pandas as pd

def get_game_info(game_pk, cache_dir="game_cache"):
    """Fetch and cache weather/game info for one game_pk"""
    os.makedirs(cache_dir, exist_ok=True)
    cache_file = f"{cache_dir}/{game_pk}.json"

    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            data = json.load(f)
    else:
        url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
        resp = requests.get(url)
        if resp.status_code != 200:
            return None
        data = resp.json()
        with open(cache_file, "w") as f:
            json.dump(data, f)
        time.sleep(0.5)

    gd = data.get("gameData", {})
    gi = gd.get("game", {})
    dt = gd.get("datetime", {})
    venue = gd.get("venue", {})
    teams = gd.get("teams", {})
    weather = gd.get("weather", {})
    game_info = gd.get("gameInfo", {})

    return {
        "game_pk": game_pk,
        "sky": weather.get("condition"),
        "temp": weather.get("temp"),
        "wind": weather.get("wind", ""),
        "gametype": gi.get("type"),
        "site": venue.get("name"),
        "hometeam": teams.get("home", {}).get("name"),
        "date": dt.get("dateTime"),
        "season": gi.get("season"),
        "daynight": dt.get("dayNight"),
        "starttime": game_info.get("firstPitch") or dt.get("time"),
        "location": venue.get("location", {}).get('city'),
        "elevation": venue.get("location", {}).get('elevation'),
        "attendance": game_info.get("attendance"),
        "capacity": venue.get("fieldInfo", {}).get('capacity'),
        "turfType": venue.get("fieldInfo", {}).get('turfType'),
        "roofType": venue.get("fieldInfo", {}).get('roofType'),
        "leftLine": venue.get("fieldInfo", {}).get('leftLine'),
        "leftCenter": venue.get("fieldInfo", {}).get('leftCenter'),
        "center": venue.get("fieldInfo", {}).get('center'),
        "rightCenter": venue.get("fieldInfo", {}).get('rightCenter'),
        "rightLine": venue.get("fieldInfo", {}).get('rightLine'),
    }

# Example: loop over a few games
game_ids = df["game_pk"].unique() #[746087, 746088]
records = []
for gid in game_ids:
    rec = get_game_info(gid)
    if rec:
        records.append(rec)

# merge back into Statcast df
games_df = pd.DataFrame(records)
df = df.merge(games_df, on="game_pk", how="left")
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,stand,...,elevation,attendance,capacity,turfType,roofType,leftLine,leftCenter,center,rightCenter,rightLine
0,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,R,...,1001.0,0,41149,Grass,Open,335,385.0,400,375.0,325
1,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,R,...,1001.0,0,41149,Grass,Open,335,385.0,400,375.0,325
2,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,L,...,1001.0,0,41149,Grass,Open,335,385.0,400,375.0,325
3,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,R,...,1001.0,0,41149,Grass,Open,335,385.0,400,375.0,325
4,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,L,...,1001.0,0,41149,Grass,Open,335,385.0,400,375.0,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122126,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,L,...,NaN,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325
122127,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,R,...,NaN,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325
122128,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,R,...,NaN,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325
122129,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,R,...,NaN,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325


In [ ]:
# convert wind str to int
df['wind_int'] = df.wind.str.split(' ').str[0]


# add barrel column
def is_barrel(row):
    ev = row['launch_speed']
    la = row['launch_angle']

    if ev < 98:
        return False

    if ev >= 116:
        return 8 <= la <= 50  # capped range
    else:
        # expansion formula
        expansion = ev - 98
        min_angle = 26 - expansion
        max_angle = 30 + expansion
        return min_angle <= la <= max_angle

df['barrel'] = df.apply(is_barrel, axis=1).astype(int)
df


,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,stand,...,attendance,capacity,turfType,roofType,leftLine,leftCenter,center,rightCenter,rightLine,barrel
0,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,R,...,0,41149,Grass,Open,335,385.0,400,375.0,325,0
1,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,R,...,0,41149,Grass,Open,335,385.0,400,375.0,325,0
2,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,L,...,0,41149,Grass,Open,335,385.0,400,375.0,325,0
3,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,R,...,0,41149,Grass,Open,335,385.0,400,375.0,325,0
4,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,L,...,0,41149,Grass,Open,335,385.0,400,375.0,325,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122126,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,L,...,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325,0
122127,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,R,...,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325,0
122128,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,R,...,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325,0
122129,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,R,...,15952,17000,Artificial Turf,Dome,325,NaN,400,NaN,325,0


In [ ]:
# sort and select columns
df = df[['game_date', 'game_pk', 'gametype', 'site', 'hometeam', 'date',
     'season', 'daynight','starttime', 'location', 'elevation', 'attendance',
     'capacity','turfType', 'roofType', 'leftLine', 'leftCenter', 'center',
       'rightCenter', 'rightLine', 'wind_int',  'sky', 'temp', 'wind',
    'batter', 'pitcher', 'release_speed', 'launch_speed', 'launch_angle', 'age_bat',
    'pitch_type', 'stand', 'p_throws', 'pitcher_batter_handedness', 'barrel', 'events', 'home_run']]
df

,game_date,game_pk,gametype,site,hometeam,date,season,daynight,starttime,location,...,launch_speed,launch_angle,age_bat,pitch_type,stand,p_throws,pitcher_batter_handedness,barrel,events,home_run
0,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,Atlanta,...,87.6,-30,35,FF,R,R,1,0,field_out,0
1,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,Atlanta,...,72.0,21,30,SL,R,R,1,0,single,0
2,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,Atlanta,...,76.1,58,30,FF,L,R,0,0,field_out,0
3,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,Atlanta,...,86.7,-25,34,FF,R,R,1,0,field_out,0
4,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,Atlanta,...,107.2,35,31,CU,L,R,0,1,home_run,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122126,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,Seoul,...,76.6,19,34,SI,L,R,0,0,single,0
122127,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,Seoul,...,59.2,-31,25,FF,R,R,1,0,field_out,0
122128,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,Seoul,...,90.9,60,32,FF,R,R,1,0,field_out,0
122129,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,Seoul,...,86.6,33,29,FF,R,R,1,0,field_out,0


In [ ]:
# save dataset
df.to_csv('baseball_clean_2024.csv')

In [ ]:
# save dataset
df.to_csv("baseball_numbers_2024.csv")

In [ ]:
df2 = pd.read_csv('baseball_numbers_2024.csv')
df2

,Unnamed: 0,game_date,game_pk,gametype,site,hometeam,date,season,daynight,starttime,...,launch_speed,launch_angle,age_bat,pitch_type,stand,p_throws,pitcher_batter_handedness,barrel,events,home_run
0,0,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,...,87.6,-30,35,FF,R,R,1,0,field_out,0
1,1,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,...,72.0,21,30,SL,R,R,1,0,single,0
2,2,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,...,76.1,58,30,FF,L,R,0,0,field_out,0
3,3,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,...,86.7,-25,34,FF,R,R,1,0,field_out,0
4,4,2024-09-30,747139,R,Truist Park,Atlanta Braves,2024-09-30T17:10:00Z,2024,day,2024-09-30T17:11:00.000Z,...,107.2,35,31,CU,L,R,0,1,home_run,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122126,122126,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,...,76.6,19,34,SI,L,R,0,0,single,0
122127,122127,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,...,59.2,-31,25,FF,R,R,1,0,field_out,0
122128,122128,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,...,90.9,60,32,FF,R,R,1,0,field_out,0
122129,122129,2024-03-20,745444,R,Gocheok Sky Dome,San Diego Padres,2024-03-20T10:05:00Z,2024,night,2024-03-20T10:07:00.000Z,...,86.6,33,29,FF,R,R,1,0,field_out,0


In [ ]:
df2.release_speed

,release_speed
0,97.4
1,90.4
2,94.2
3,95.2
4,86.4
...,...
122126,93.9
122127,96.6
122128,96.9
122129,94.5
